In [ ]:
# Google Drive マウント
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/卒業研究/DeadReckoning_ExperimentalData/ML'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/卒業研究/DeadReckoning_ExperimentalData/ML
data/                ML_angle.ipynb      result_ML_MultiData.csv
DeadReckoning.ipynb  ML.ipynb            result_ML_MultiData.gsheet
graph/               ML_MultiData.ipynb  scripts/
loaddata.ipynb       model/              Untitled0.ipynb
logs/                others/
makedata.ipynb       __pycache__/


In [ ]:
# !pip install --upgrade tensorflow

In [ ]:
!pip install tqdm

In [ ]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

plt.style.use('ggplot')

In [ ]:
# def load_data(filename):
#     data = np.loadtxt(filename, delimiter=',', skiprows=1,
#                       unpack=False, dtype=str)
#     speed = data[:, 1].astype('f8')
#     # speed = speed[:, np.newaxis]
#     acc = data[:, 2: 2+3*50]
#     return acc, speed

def load_data_vector(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1,
                      unpack=False, dtype=str)
    speed = []
    xyzwave = []
    for row in data:
        speed.append(row[1].astype('f8'))
        xyzwave.append(row[2: 2 + 3*50].astype('f8'))
    speed = np.array(speed)
    x = np.array(xyzwave)
    t = speed[:, np.newaxis]
    return x, t

In [ ]:
def load_data_norm(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1,
                      unpack=False, dtype=str)
    time = data[:, 0].astype('f8')
    speed = data[:, 1].astype('f8')
    speed = speed[:, np.newaxis]
    xyzwave_list = []
    for row in data:
        xwave = row[2:52].astype('f8')
        ywave = row[52:102].astype('f8')
        zwave = row[102:152].astype('f8')
        xyzwave = xwave**2 + ywave**2 + zwave**2
        xyzwave_list.append(xyzwave)
    xyzwave_list = np.asarray(xyzwave_list)
    return xyzwave_list, speed

In [ ]:
# ハイパーパラメータ
activation = 'relu'
activation_output = 'sigmoid'
learning_rate = 0.001
batch_size = 32
optimizer = 'adam'
loss = 'mse'

In [ ]:
import keras.backend as K
K.clear_session()

# モデル生成
def vector_model():
    vector_model = keras.models.Sequential([
        layers.Input(shape=(150,)),
        layers.Dropout(0.2),
        layers.Dense(100, activation=activation),
        layers.Dense( 50, activation=activation),
        layers.Dense( 25, activation=activation),
        layers.Dense( 12, activation=activation),
        layers.Dense(  1, activation=activation_output)
    ])

    adam = keras.optimizers.Adam(lr=learning_rate)
    vector_model.compile(optimizer=adam, loss=loss, metrics=['mae'])
    # vector_model.summary()
    return vector_model

In [ ]:
import keras.backend as K
K.clear_session()

# モデル生成
def norm_model():
    norm_model = keras.models.Sequential([
        layers.Input(shape=(50,)),
        layers.Dense(100, activation=activation),
        layers.Dropout(0.2),
        layers.Dense( 50, activation=activation),
        layers.Dense( 25, activation=activation),
        layers.Dense( 12, activation=activation),
        layers.Dense(  1, activation=activation_output)
    ])

    adam = keras.optimizers.Adam(lr=learning_rate)
    norm_model.compile(optimizer=adam, loss=loss)
    # norm_model.summary()
    return norm_model

In [ ]:
from tqdm import tqdm

def mse_test(model_fanc, x, t):
    SPEED_MAX = 2.5
    t /= SPEED_MAX

    mae_train_list = []
    mae_test_list = []
    for i in tqdm(range(5)):
        x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.1)

        # model.reset_states()
        model = model_fanc()
        result = model.fit(x_train, t_train, epochs=100,
                    batch_size=batch_size, verbose=0,
                    validation_data=(x_test, t_test))

        y_train = model.predict(x_train)
        t_train *= SPEED_MAX
        y_train *= SPEED_MAX
        mae_train = mean_absolute_error(t_train, y_train) * 100
        mae_train_list.append(mae_train)

        y_test = model.predict(x_test)
        t_test *= SPEED_MAX
        y_test *= SPEED_MAX
        mae_test = mean_absolute_error(t_test, y_test) * 100
        mae_test_list.append(mae_test)
        # plot_result(result, t_train, y_train, t_test, y_test)
    return mae_train_list, mae_test_list
    # print(mae_train_list, mae_test_list)
    # return mae_train_list[0], mae_test_list[0]

paths = [
    'data/191121_1625/ML.csv',
    'data/191106_1152/ML.csv',
    'data/191204_1956/ML.csv',
    'data/191204_2012/ML.csv',
    'data/191211_1649/ML.csv',
]

df_result_mean = pd.DataFrame()

for path in paths:
    x, t = load_data_vector(path)
    mae_train_list_vector, mae_test_list_vector = mse_test(vector_model, x, t)

    x, t = load_data_norm(path)
    mae_train_list_norm, mae_test_list_norm = mse_test(norm_model, x, t)

    df_result = df_result.append({'path': path, 
                                  'mae_train_vector_mean': np.mean(mae_train_list_vector), 
                                  'mae_train_vector_std': np.std(mae_train_list_vector),
                                  'mae_test_vector_mean': np.mean(mae_test_list_vector), 
                                  'mae_test_vector_std': np.std(mae_test_list_vector), 
                                  'mae_train_norm_mean': np.mean(mae_train_list_norm), 
                                  'mae_train_norm_std': np.std(mae_train_list_norm),
                                  'mae_test_norm_mean': np.mean(mae_test_list_norm), 
                                  'mae_test_norm_std': np.std(mae_test_list_norm),}, 
                                 ignore_index=True)

100%|██████████| 5/5 [00:35<00:00,  7.06s/it]


In [ ]:
df_result.to_csv('result_ML_MultiData.csv')

df_result

,mae_test_norm_mean,mae_test_norm_std,mae_test_vector_mean,mae_test_vector_std,mae_train_norm_mean,mae_train_norm_std,mae_train_vector_mean,mae_train_vector_std,path
0,7.131143,0.625631,6.728761,0.279049,5.119432,0.195938,5.424726,0.318705,data/191121_1625/ML.csv
1,9.539870,0.943061,7.745110,0.365213,5.996680,0.224700,3.706597,0.186606,data/191106_1152/ML.csv
2,6.975276,0.665220,7.314438,0.963583,4.337340,0.310595,4.814254,0.426013,data/191204_1956/ML.csv
3,7.721694,0.929134,8.188355,1.202981,5.479600,0.291575,5.344427,0.337979,data/191204_2012/ML.csv
4,4.644412,0.710122,5.798639,0.807145,3.436515,0.301213,4.492735,0.329287,data/191211_1649/ML.csv
5,7.066562,0.254972,7.160027,0.408098,5.332047,0.236431,5.466235,0.271717,data/191121_1625/ML.csv
6,7.779188,0.750530,8.354618,1.338064,3.739990,0.384971,2.778119,0.529473,data/191106_1152/ML.csv
7,7.202446,0.601776,7.829437,0.753823,5.026515,0.800630,5.772630,0.576018,data/191204_1956/ML.csv
8,6.872159,1.278543,8.605422,1.045920,5.659198,0.406886,5.904393,0.375632,data/191204_2012/ML.csv
9,4.344710,0.151624,4.800896,0.568464,3.701642,0.403583,4.252008,0.429495,data/191211_1649/ML.csv


In [ ]:
df_result.describe()

,mae_test_norm_mean,mae_test_norm_std,mae_test_vector_mean,mae_test_vector_std,mae_train_norm_mean,mae_train_norm_std,mae_train_vector_mean,mae_train_vector_std
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,6.927746,0.691061,7.252570,0.773234,4.782896,0.355652,4.795612,0.378093
std,1.498356,0.327160,1.197552,0.365420,0.910969,0.173127,0.998582,0.116684
min,4.344710,0.151624,4.800896,0.279049,3.436515,0.195938,2.778119,0.186606
25%,6.897938,0.607739,6.836577,0.448189,3.889328,0.250217,4.312190,0.321351
50%,7.098852,0.687671,7.529774,0.780484,5.072974,0.305904,5.079340,0.356805
75%,7.591882,0.884483,8.098625,1.025336,5.442712,0.398930,5.455858,0.428625
max,9.539870,1.278543,8.605422,1.338064,5.996680,0.800630,5.904393,0.576018


In [ ]:
def plot_result(result, t_train, y_train, t_test, y_test):
    fig = plt.figure(figsize=(10, 10))

    ax1 = fig.add_subplot(311)
    # ax1.set_title('activation={}, '.format(activation) + 
    #                 'activation_output={}, \n'.format(activation_output) + 
    #                 'learning_rate={}, '.format(learning_rate) +
    #                 'optimizer={}, '.format(optimizer) + 
    #                 'loss={}'.format(loss))
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('loss')
    ax1.plot(result.history['loss'], color='tab:blue', label='loss')
    ax1.plot(result.history['val_loss'], color='tab:green', label='val_loss')
    ax1.legend(loc='upper right')

    ax2 = fig.add_subplot(312)
    mae_train = mean_absolute_error(t_train, y_train)
    ax2.set_title('train data \n mean absolute error = {0:.3} [m/s]' \
                  .format(mae_train))
    ax2.set_xlabel('time [s]')
    ax2.set_ylabel('speed [m/s]')
    ax2.plot(t_train, color='tab:gray', label='RTK')
    ax2.plot(y_train, color='tab:blue', label='NN', alpha=0.9)
    ax2.legend(loc='upper left')

    ax3 = fig.add_subplot(313)
    mae_test = mean_absolute_error(t_test, y_test)
    ax3.set_title('test data \n mean absolute error = {0:.3} [m/s]' \
                  .format(mae_test))
    ax3.set_xlabel('time [s]')
    ax3.set_ylabel('speed [m/s]')
    ax3.plot(t_test, color='tab:gray', label='RTK')
    ax3.plot(y_test, color='tab:green', label='NN', alpha=0.9)
    ax3.legend(loc='upper left')

    fig.align_labels()
    fig.subplots_adjust(hspace=0.6)
    # fig.savefig('graph/graph.png')

# 学習結果
# plt.rcParams['font.size'] = 11
# plot_result()

In [ ]:
def test(filename: str):
    x, t = load_data(filename)
    y = model.predict(x) * SPEED_MAX
    return t, y

def plot_test(t, y):
    fig = plt.figure(figsize=(7.5, 5))
    ax = fig.add_subplot(111)
    mae = mean_absolute_error(t, y)
    # ax.set_title('mean absolute error = {:.3} [m/s]'.format(mae))
    ax.set_xlabel('time [s]')
    ax.set_ylabel('speed [m/s]')
    ax.plot(t, color='gray', label='RTK')
    ax.plot(y, color='tab:blue', label='NN', alpha=0.7)
    ax.legend(loc='upper left')
    print(len(t))

In [ ]:
t, y = test('data/191121_1625/ML.csv')
plot_test(t, y)

t, y = test('data/191106_1152/ML.csv')
plot_test(t[10:-5], y[10:-5])

t, y = test('data/191204_1956/ML.csv')
plot_test(t, y)

t, y = test('data/191204_2012/ML.csv')
plot_test(t, y)

t, y = test('data/191211_1649/ML.csv')
plot_test(t[15:-5], y[15:-5])

NameError: ignored